<img src="../../share/skience2020_logo.png"
     alt="Markdown Monster icon"
     width="300"
     style="float: right; margin-right: 100px;" />
     
## Adjoint Method

##### Author:
* Sebastian Noe (email: snoe@geophysik.uni-muenchen.de)

---

### Literature

* Bradley, A. M. (2019). PDE-constrained optimization and the adjoint method. Stanford University.  
* Fichtner, A., Bunge, H.-P., and Igel, H. (2006a). The adjoint method in seismology  Physics of the Earth and Planetary Interiors, 157(1-2):86–104.  
* Fichtner, A., Bunge, H.-P., and Igel, H. (2006b). The adjoint method in seismology Physics of the Earth and Planetary Interiors, 157(1-2):105–123.  
* Menke, B. (2007). A tutorial on adjoint methods of calculating data kernels. Columbia University
* Plessix, R.-E. (2006). A review of the adjoint-state method for computing the gradient of a functional with geophysical applications. Geophysical Journal International, 167(2):495–503.  
* Tromp, J., Tape, C., and Liu, Q. (2005). Seismic tomography, adjoint methods, time reversal and banana-doughnut kernels. Geophysical Journal International, 160(1):195–216. 

***

### Summary

With the adjoint method the model derivatives of an objective function can be exactly calculated where the objective function quantifies the misfit between observed and synthetic data sets. Hence, it is a useful tool for inverse problems as one tries to find the model that minimizes the objective function. 
If it is possible to model the physical problem with a partial differential equation (e.g. wave equation), the model derivatives of the objective function can be computed by solving the forward problem twice. Derivations expanding the volume of this notebook can be found in _Fichtner et al._ (2006a).

***

### Introduction -- What's an adjoint?

Let's define an _inner product_ as the integral over the product of two real-valued functions:

$
\langle f(t),g(t)\rangle := \int_{t_0}^{t_1}f(t)g(t)dt
$

Functions can be subject to a differential operator $L$ , e.g. $L=\partial_t$, $L=\rho\partial^2_t-\nabla\cdot(\mu\nabla)$. <br>
One could define the adjoint $L^*$ as the differential operator that satisfies

$
\langle f,Lg\rangle = \langle g,L^*f\rangle
$

If you have trouble imagining an adjoint, it's possible to draw an analogy to vectors and dot products. We can define a dot product for two real-valued vectors of the same length:

$
v \cdot w = \sum_i v_iw_i
$

Vectors can be subject to matrices $M$. The transponse of a matrix has following relationship in combination with a dot product:

$
u\cdot(Mw) = w\cdot (M^Tu)
$

_Basically, an adjoint is to an operator what a transpose is to a matrix._

***
__Examples -- Adjoints of first and second derivatives__ 


$L = \partial_t \rightarrow L^* = -\partial_t$ <br>

_Proof:_  

$\langle f,Lg\rangle=\langle f,\partial_tg\rangle = \int_{t_0}^{t_1}f\partial_tg~dt \stackrel{(1)}{=} fg\vert_{t_0}^{t_1}-\int_{t_0}^{t_1}\partial_tfgdt \stackrel{(2)}{=} \langle g,-\partial_tf\rangle=\langle g,-Lf\rangle$ 

(1) partial integration   
(2) assume homogeneous boundary conditions, i.e. $\bigl(f(t_0)=0 \lor g(t_0)=0\bigr) \land \bigl(f(t_1)=0 \lor g(t_1)=0\bigr)$ 


It is easy to show that the second derivative is _self-adjoint_ 

$L = \partial^2_t \rightarrow L^* = \partial^2_t  = L$  

_Proof:_ 

$\langle f,Lg\rangle=\langle f,\partial^2_tg\rangle = \int_{t_0}^{t_1}f\partial^2_tg~dt \stackrel{(1)}{=}  f\partial_tg\vert_{t_0}^{t_1}- \partial_tfg\vert_{t_0}^{t_1}+\int_{t_0}^{t_1}g\partial^2_tfdt \stackrel{(2)}{=} \langle g,\partial^2_tf\rangle = \langle g,Lf\rangle $ 

(1) partial integration (twice)  
(2) homogeneous boundary conditions for $f,g,\partial_t f,\partial_t g$

Note that the introduction of certain boundary conditions is crucial for the existence of an adjoint.

***

### Adjoint Method -- general formulation (after Fichtner et al. 2006a)

Let's look at a physical problem described by the partial differential equation 

$ L(u;\vec{x},\vec{m},t) = g(\vec{x},t)$ 

and an objective function

$\zeta=\int_{t_0}^{t_1}\int_G f(u,\vec{m}) d\vec{x}dt = \langle 1,f\rangle$

where: <br>
$L$ -- differential operator <br>
$g(\vec{x},t)$ -- source term <br>
$u(\vec{x},t)$ -- observable, constrained by boundary conditions<br>
$t$ -- time<br>
$\vec{m}$ -- model parameters <br>
$\vec{x}$ -- location in G<br>
$G$ -- spatial domain<br>
$[t_0,t_1]$ -- observation interval<br>
$f$ -- function quantifying the misfit at observation locations<br>

What happens when we try to calculate the (total) derivative of the objective function in a straightforward way? 

$
d_{m_i}\zeta(u,\vec{m}) = \langle 1,d_{m_i}f(u,\vec{m})\rangle \stackrel{(1)}{=} \langle 1, (\partial_uf)(d_{m_i}u) + \partial_{m_i}f\rangle = \langle d_{m_i}u,\partial_uf\rangle + \langle 1,\partial_{m_i}f\rangle
$

(1) Chain rule  

Here, the total derivative of the observable with respect to the model parameters is a problem. There is no easy way to calculate it. The aim of the adjoint method is to somehow eliminate $d_{m_i}u$ from the term for the total derivative of the objective function.  
In order to archieve this, we look at the partial differential equation describing our physical problem $L=g$ and apply total model derivatives to both sides. 

$
(\partial_uL)d_{m_i} u + \partial_{m_i} L = 0
$

As the source $g(x,t)$ is model independent, we get zero on the right hand side. This means we can multiply the left hand side by an arbitrary test function $\Psi(x,t)$ (defined on $G$ and $t\in[t_0,t_1]$, same as $u$) and the right hand side is still zero.

$
\langle \Psi, (\partial_uL)d_{m_i} u \rangle +\langle \Psi, \partial_{m_i} L \rangle = 0
$

Applying the definition of an adjoint we get:

$
\langle d_{m_i} u,(\partial_uL)^*\Psi\rangle+\langle 1, (\partial_{m_i}L)^*\Psi\rangle = 0
$

As the right hand side is zero, we can add the quantity of the left hand side to the total derivative of the objective function without changing it's value. Rearranging the integrals yields

$
d_{m_i}\zeta = \langle d_{m_i}u, (\partial_uL)^*\Psi + \partial_uf \rangle + \langle 1, (\partial_{m_i}L)^*\Psi +\partial_{m_i}f\rangle
$

This result is crucial. We can eliminate $d_{m_i}u $ from the equation by choosing a test function $\Psi$ that satisfies

$(\partial_u L)^*\Psi +\partial_u f = 0$

This is a partial differential equation and the so-called **Adjoint Equation** where $\Psi$ is the adjoint observable.
With knowledge of $\Psi$, one could calculate the gradient of objective function for model parameters $m_i$ with the second term

$d_{m_i}\zeta =\langle 1,(\partial_{m_i}L)^*\Psi+\partial_{m_i}f\rangle$ 

The determination of the adjoints of $\partial_uL$ and $\partial_{m_i}L$ leads to boundary conditions on the adjoint observable. 
Adjoint equation and subsidiary conditions combined are the adjoint problem.


***

###  Adjoint problem applied to the 1D scalar wave equation

On a 1D string, the scalar wave equation describes the wavefield u(x,t) due to a source term g(x,t) depending on the model parameters density $\rho$ and shear modulus $\mu$. Hence,

$L(u;x,\vec{m},t) = \rho(x)\partial_t^2 u(x,t) - \partial_x(\mu(x)\partial_xu(x,t)) = g(x,t)$

where $\vec{m}=\begin{pmatrix}\rho\\ \mu\end{pmatrix}$.

The wavefield $u$ is subject to the following conditions:

I - Initial condition:  
$
\partial_t u(x,t_0) = u(x,t_0) = 0
$

II - Cauchy condition:  
$
u(0,t) = u(x_{max},t) = 0
$

III - Neumann condition:  
$
\partial_xu(0,t) = \partial_xu(x_{max},t) = 0
$



A common way to measure the misfit between observed and sythetic wavefields is with a L2-norm objective function: 

$
\zeta = \int_{t_0}^{t_1}\int_{0}^{x_{max}} f(u) dxdt
$

where 

$
f(u) = \frac{1}{2}[u(x,t)-u_0(x,t)]^2\delta(x-x_r)
$

with $u_0$ the wavefield as it was observed at stations $x_r$.

***
The partial derivatives of the operator $L$ are

$\partial_u L=\rho\partial_t^2 - \partial_x(\mu\partial_x)$  
$\partial_\rho L=\partial_t^2u$   
$\partial_\mu L=-\partial^2_xu$  

We have shown that the second derivatives are self-adjoint. In fact, it's possible to show that above operators are all self-adjoint

$\partial_u L^*=\rho\partial_t^2 - \partial_x(\mu\partial_x)$  
$\partial_\rho L^*=\partial_t^2u$   
$\partial_\mu L^*=-\partial^2_xu$ 

if following conditions are applied to adjoint wavefield $\Psi$:

I - Terminal condition:  
$
\partial_t \Psi(x,t_1) = \Psi(x,t_1) = 0
$

II - Cauchy condition:  
$
\Psi(0,t) = \Psi(x_{max},t) = 0
$

III - Neumann condition:  
$
\partial_x\Psi(0,t) = \partial_x\Psi(x_{max},t) = 0
$

Thus the **Adjoint Equation** takes the following form

$
\rho\partial_t^2 \Psi - \partial_x(\mu\partial_x\Psi) = -\partial_u f
$

We note that the adjoint wavefield $\Psi$ is subject to the same operator than the forward wavefield $u$. This result is obtained because we have chosen a self-adjoint differential operator at the beginning. This is not always the case. For example, one could modify the operator to include first derivatives (e.g. consider attenuation), which will lead to an adjoint wavefield that doesn't behave exactly like the originial one. However, even in our non-damped system there are two noticable differences between adjoint and original wavefields. First, the source term for the adjoint problem is the misfit at each individual station. In other words, the misfit of our synthetic data to the observed data is redirected into the model as energy at each receiver, which creates the adjoint wave field. Second, the adjoint wavefield satisfied terminal rather than initial conditions. 
***

The goal of the adjoint method is to calculate the derivative of the objective function $\zeta$ with respect to the model parameters $\rho$ and $\mu$.  
We insert our findings in the given general equations

$
d_{\rho}\zeta = \langle 1, \partial_{\rho}f+(\partial_t^2u)\Psi \rangle
$   
$
d_\mu\zeta = \langle 1, \partial_\mu f -(\partial^2_xu)\Psi \rangle
$  

As f is independent of the model parameters it's partial derivatives disappear. We could utilize the subsidiary conditions in a sophisticated way to arrive at model derivatives that are symmetric in original and adjoint wavefields.

$
d_\rho\zeta = \int_{t_0}^{t_1}\int_G (\partial_t u)(\partial_t \Psi)dxdt
$  
$
d_\mu\zeta = -\int_{t_0}^{t_1}\int_G (\partial_x u)(\partial_x \Psi)dxdt
$

Note that for this calculation the forward model must be solved twice, once for the original wavefield and once for the adjoint wavefield. With the knowledge of the model derivatives of the objective function, we know how to update the model for the synthetic data set. With the means of an iterative scheme the inverse problem can be solved. 
***
Let's simulate previous 1D example of the scalar wave equation.

In [ ]:
## Cell 0 - import libraries
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from IPython.display import clear_output

In [ ]:
## Cell 1 
# Model set up
x_max = 10000
t_max = 2
mu0 = 60e9 
rho0 = 3500 # kg/m³
c = np.sqrt(mu0/rho0)
print('Velocity: ',c,'m/s')
dx = 10
dt = dx/c
nx = int(x_max//dx)
nt = int(t_max//dt)



receiver_locations = []
receiver_position = []
rec_dis = False
if rec_dis: # Receivers distributed along string with constant offset
    N_rec = 3
    for i in range(0,N_rec):
        receiver_locations.append(int((i+1)*nx/(N_rec+1)))
        receiver_position.append(receiver_locations[i]*dx)

else: # Individual configuration of receiver position
    receiver_locations = [.3] #locations w.r.t. total length
    N_rec = len(receiver_locations)
    for i in range(0,N_rec):
        receiver_locations[i] = int((x_max//dx)*receiver_locations[i])
        receiver_position.append(receiver_locations[i]*dx)
    

x = np.linspace(0,nx,nx)*dx
t = np.linspace(0,nt,nt)*dt

First, we need to get an "observed" data set $u_0$ for a homogeneous background model with a perturbation at a single cell.

In [ ]:
## Cell 2
u0 = np.zeros((nx,nt))

isnap = 20
isrc = 1
f0   = c/(10*dx) # dominant frequency of the source (Hz)
t0   = 4./f0 # source time shift
src = np.zeros((nx,nt))
src[isrc,:]  = -2. * (t - t0) * (f0 ** 2) * (np.exp(-1.0 * (f0 ** 2) * (t - t0) ** 2))

 
rho = np.ones(nx)*rho0  
mu = np.ones(nx)*mu0

perturbation_location = .5 # w.r.t. total length
rho[int(perturbation_location*x_max)//dx] *= 1.3
#mu[int(perturbation_location*x_max)//dx] *= 0.8




### Creating observed data set 

for i in range(1,nt-1):
    for n in range(1,nx-1):
        u0[n,i+1] = dt**2/rho[n] * ( src[n,i]+\
                       u0[n,i]*(2*rho[n]/dt**2-2*mu[n]/dx**2) - u0[n,i-1]*rho[n]/dt**2 \
                    -  u0[n+1,i]*((mu[n-1]-mu[n+1])/(4*dx**2)-mu[n]/dx**2) \
                    -  u0[n-1,i]*((-mu[n-1]+mu[n+1])/(4*dx**2)-mu[n]/dx**2))

    if i%isnap==1:
        plt.figure(figsize=(8,6))
        plt.title('Observed wavefield, time: '+str(round(i*dt,1))+' s')
        plt.plot(x,u0[:,i])
        plt.scatter(receiver_position,np.zeros(N_rec),color='black',s=30,marker='v',label='Receivers')
        plt.scatter(int(perturbation_location*x_max),0,s=50,color='green',marker='x',label='Perturbation')
        plt.scatter(isrc*dx,0,color='red',marker='d',label='Source')
        
        plt.yticks([])
        plt.ylabel('Displacement')
        plt.xlabel('Position [m]')
        plt.legend(loc=1)
        clear_output(wait=True)
        plt.show() 

The synthetic dataset is created without taking the perturbation into account.

In [ ]:
## Cell 3
u = np.zeros((nx,nt))
rho = np.ones(nx)*rho0
mu = np.ones(nx)*mu0

### Creating synthetic data set for homogeneous model 

for i in range(1,nt-1):
    for n in range(1,nx-1):
        u[n,i+1] = dt**2/rho[n] * ( src[n,i]+\
                       u[n,i]*(2*rho[n]/dt**2-2*mu[n]/dx**2) - u[n,i-1]*rho[n]/dt**2 \
                    -  u[n+1,i]*((mu[n-1]-mu[n+1])/(4*dx**2)-mu[n]/dx**2) \
                    -  u[n-1,i]*((-mu[n-1]+mu[n+1])/(4*dx**2)-mu[n]/dx**2))

    if i%(2*isnap)==1:
        plt.figure(figsize=(8,6))
        plt.title('Synthetic wavefield, time: '+str(round(i*dt,1))+' s')
        plt.plot(x,u[:,i],color='orange')
        plt.scatter(isrc*dx,0,color='red',marker='d',label='Source')
        plt.scatter(receiver_position,np.zeros(N_rec),color='black',s=20,marker='v',label='Receivers')
        plt.yticks([])
        plt.ylabel('Displacement')
        plt.xlabel('Position [m]')
        plt.legend(loc=1)
        clear_output(wait=True)
        plt.show()   

Each receiver has recorded an observation and a synthetic wavefield was calculated. Both displacements can be compared with the L2-norm.

In [ ]:
# Cell 4
zeta = 0
k = 1
for xr in receiver_locations:
    
    for i in range(0,nt):
        zeta += 0.5 * (u[xr,i]-u0[xr,i])**2
    
    
    plt.figure(figsize=(8,6))
    plt.title('Receiver #'+str(k) +' at '+str(round(xr*dx))+' m')
    plt.plot(t,u[xr,:],color='orange',label='synthetic')
    plt.plot(t,u0[xr,:],label='observed')
    plt.xlabel('Time [s]')
    plt.ylabel('Displacement')
    plt.yticks([])
    plt.legend(loc=1)
    plt.show() 
    k += 1


In [ ]:
# Cell 5
print('Objective function value :')
print(zeta)

Let's move on to the adjoint problem. 
The adjoint wavefield is subject to the same differential operator as the original wavefield (because our operator is self-adjoint) and the adjoint sources are the misfits between synthetic and observed datasets at each receiver.

So each receiver turns into a source now.
Note that time runs backwards in the adjoint simulation!


In [ ]:
## Cell 6
psi = np.zeros((nx,nt))
rho = np.ones(nx)*rho0
mu = np.ones(nx)*mu0

src = np.zeros((nx,nt))
for xr in receiver_locations:
    src[xr,:]= u0[xr,:]-u[xr,:]

### Adjoint wavefield

for i in range(1,nt-1):
    for n in range(1,nx-1):
        psi[n,i+1] = dt**2/rho[n] * ( src[n,-i]+\
                       psi[n,i]*(2*rho[n]/dt**2-2*mu[n]/dx**2) - psi[n,i-1]*rho[n]/dt**2 \
                    -  psi[n+1,i]*((mu[n-1]-mu[n+1])/(4*dx**2)-mu[n]/dx**2) \
                    -  psi[n-1,i]*((-mu[n-1]+mu[n+1])/(4*dx**2)-mu[n]/dx**2))

for i in range(0,nt):        
    if i%isnap==0 or i==(nt-1):
        plt.figure(figsize=(8,6))
        plt.title('Adjoint wavefield, time: '+str(round((nt-i)*dt,1))+' s')
        plt.plot(x,psi[:,i],color='green')
        plt.scatter(receiver_position,np.zeros(N_rec),color='red',s=20,marker='d',label='Adjoint sources')
        plt.scatter(int(perturbation_location*x_max),0,color='black',s=50,marker='x',label='Perturbation')
        plt.yticks([])
        plt.ylabel('Displacement')
        plt.xlabel('Position [m]')
        plt.legend(loc=1)
        plt.ylim((max(psi[nx//2,:]),-max(psi[nx//2,:])))
        clear_output(wait=True)
        plt.show()

And we can combine both wavefields.

In [ ]:
## Cell 7
scaling = max(psi[nx//2,:])/max(u[nx//2,:])*1.5
for i in range(0,nt):
    if i%isnap==1 or i==(nt-1):
        plt.figure(figsize=(8,6))
        plt.title('Synthetic and adjoint wavefields, time: '+str(round(i*dt,1))+' s')
        plt.plot(x,u[:,i]*scaling,color='blue',label='synth. wavefield')
        plt.plot(x,psi[:,-i],color='green',label='adjoint wavefield')
        plt.scatter(int(perturbation_location*x_max),0,s=50,color='black',marker='x',label='Perturbation')
        plt.scatter(isrc*dx,0,color='red',marker='d',label='Source',s=20)
        plt.scatter(receiver_position,np.zeros(N_rec),color='black',s=20,marker='v',label='Adjoint Sources')
        plt.yticks([])
        plt.ylabel('Displacement')
        plt.xlabel('Position [m]')
        plt.legend(loc=1)
        clear_output(wait=True)
        plt.show()

We have solved the adjoint problem. Thus, the objective function's derivatives can be calculated.

In [ ]:
## Cell 8
dt_u = np.zeros((nx,nt))
dt_psi = np.zeros((nx,nt))
dx_u = np.zeros((nx,nt))
dx_psi = np.zeros((nx,nt))

dzeta_drho = np.zeros(nx)
dzeta_dmu = np.zeros(nx)

for i in range(1,nt-1):
    dt_u[:,i] = (u[:,i+1]-u[:,i-1])/(2*dt) 
    dt_psi[:,i] = (psi[:,i]-psi[:,i-1])/(2*dt)
        
for n in range(1,nx-1):
    dx_u[n,:] = (u[n+1,:]-u[n-1,:])/(2*dx) 
    dx_psi[n,:] = (psi[n,:]-psi[n-1,:])/(2*dx)    
    
    
for i in range(0,nt):
    dzeta_drho[:] -= dt_u[:,i] * dt_psi[:,-i]
    dzeta_dmu[:] += dx_u[:,i] * dx_psi[:,-i]
    
plt.title('Derivative w.r.t. density')    
plt.plot(x,dzeta_drho[:])
plt.scatter(int(perturbation_location*x_max),dzeta_drho[int(perturbation_location*x_max)//dx],s=50,color='black',marker='x',label='Perturbation')
#plt.yticks([])
plt.xlabel('Position [m]')
plt.show()


plt.title('Derivative w.r.t. shear modulus')
plt.plot(x,dzeta_dmu[:])
plt.xlabel('Position [m]')
plt.scatter(int(perturbation_location*x_max),dzeta_dmu[int(perturbation_location*x_max)//dx],s=50,color='black',marker='x',label='Perturbation')
#plt.yticks([])
plt.show()



You can easily vary total simulation time, number of receivers, receiver locations (Cell 1), perturbation location and perturbation strength (Cell 2). 